# Fundamentos de la Programación (Grado en Ingeniería de la Salud)
## Ejercicio "Películas" (11 de diciembre de 2019)

Antes de empezar, ejecutaremos la siguiente celda con las importaciones necesarias para poder realizar el ejercicio:

In [1]:
import csv
from datetime import datetime
from matplotlib import pyplot as plt
from collections import namedtuple

## 1. Carga de datos (0.5 puntos)

Tomaremos los datos de un fichero de entrada llamado <code>peliculas.csv</code> en el que se encuentran registrados datos sobre una serie de películas. El fichero se encuentra en la carpeta <code>./data</code>. Cada línea del fichero de entrada contiene las siguientes informaciones:

- **title:** título de la película
- **genres:** géneros en los que se cataloga la película
- **popularity:** grado de popularidad de la película
- **release_date:** fecha de estreno de la película

He aquí un fragmento con las primeras líneas del fichero de entrada:

<img src="./img/datos.png" alt="Drawing" style="width:14cm;"/>


La primera función que implementaremos será la de lectura. Será la encargada de leer los datos del fichero de entrada y cargarlos en una lista de tuplas:

In [ ]:
Pelicula = namedtuple('Pelicula', 'titulo generos popularidad fecha')

def lee_peliculas(fichero):
    ''' Lee el fichero de entrada y devuelve una lista de registros
    
    ENTRADA:
       - fichero: nombre del fichero de entrada
    SALIDA:
       - lista de registros -> [Pelicula(str, str, float, datetime)]

    Hay que transformar ciertos elementos de la entrada en valores numéricos o de otros tipos
    para que puedan ser procesados posteriormente.
    
    Para convertir una cadena con el formato DD/MM/AAAA en un objeto 'date' usaremos
    la siguiente expresión:
    
        datetime.strptime(fecha,'%d/%m/%Y')
    '''
    registros = []
    with open(fichero, 'r', encoding='utf-8') as f:
        lector = csv.reader(f)
        next(lector)
        registros = [Pelicula(titulo, generos, float(popularidad), datetime.strptime(fecha,'%d/%m/%Y'))
                    for titulo, generos, popularidad, fecha in lector]
    return registros

In [ ]:
# TEST de 'lee_peliculas', la salida esperada es:
# 15698 [Pelicula(titulo='Ariel', generos='Drama|Crime', popularidad=0.8239040000000001, fecha=datetime.datetime(1988, 10, 21, 0, 0)), 
#        Pelicula(titulo='Four Rooms', generos='Crime|Comedy', popularidad=1.6980000000000002, fecha=datetime.datetime(1995, 12, 25, 0, 0))]
REGISTROS = lee_peliculas('./data/peliculas.csv')
print(len(REGISTROS), REGISTROS[:2])

## 2. Consulta y filtrado (7.5 puntos)

Una vez que hemos cargado los datos en una estructura en memoria ya podemos empezar a procesarlos. En esta sección implementaremos algunas funciones de consulta y filtrado que nos permitirán trabajar con ellos.

La primera función se llama <code>calcula_generos</code> y calcula un conjunto de géneros presentes en la colección de datos:

In [ ]:
def calcula_generos(registros):
    ''' Conjunto de géneros presentes en una lista de registros
    
    ENTRADA:
       - registros: lista de registros ->  [Pelicula(str, str, float, datetime)]
    SALIDA:
       - términos que aparecen al menos una vez en el campo géneros
    '''
    generos = set()
    for r in registros:
        generos_r = set(r.generos.split('|'))
        generos |= generos_r
    return generos

In [ ]:
# TEST de 'calcula_generos', la salida esperada es:
#    ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 
#     'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 
#     'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']

print(sorted(calcula_generos(REGISTROS)))

La siguiente función es <code>filtra_por_años</code>. La función toma una lista de registros y selecciona solo aquellos que se corresponden con películas estrenadas en el rango de fechas indicado:

In [ ]:
def filtra_por_años(registros, año_inicio=None, año_fin=None):
    ''' Películas estrenadas en el rango de años indicado (ambos incluidos)
    
    ENTRADA:
       - registros: lista de registros ->  [Pelicula(str, str, float, datetime)]
       - año_inicio: año inicial del rango (None indica que el año inicial es el menor de la colección) -> int
       - año_fin: año final del rango (None indica que el año final es el mayor de la colección) -> int
    SALIDA:
       -  lista de registros seleccionados -> [Pelicula(str, str, float, datetime)]
       
    Para obtener el año de un objeto fecha, se puede usar el campo '.year'
    '''
    años = {r.fecha.year for r in registros}
    if año_inicio is None:
        año_inicio = min(años)
    if año_fin is None:
        año_fin = max(años)
    return [r for r in registros if año_inicio<=r.fecha.year<=año_fin]

In [ ]:
# TEST de 'filtra_por_años', la salida esperada es:
#   4866
#   9978
#  10586
#  10586
#      8

print(len(filtra_por_años(REGISTROS, 2000, 2010)))
print(len(filtra_por_años(REGISTROS, año_inicio=2000)))
print(len(filtra_por_años(REGISTROS, año_fin=2010)))
print(len(filtra_por_años(REGISTROS, año_fin=2010)))
print(len(filtra_por_años(REGISTROS, año_inicio=2020)))

La siguiente función es <code>filtro_generico</code>. La función toma una lista de registros y selecciona solo aquellos para los que el criterio recibido como parámetro se cumple:

In [ ]:
def filtro_generico(registros, criterio):
    ''' Películas que cumplen el criterio recibido como parámetro
    
    ENTRADA:
       - registros: lista de registros ->  [Pelicula(str, str, float, datetime)]
       - criterio: nombre de función o expresión lambda -> function
    SALIDA:
       -  lista de registros seleccionados -> [Pelicula(str, str, float, datetime)]
    '''
    return [r for r in registros if criterio(r)]

In [ ]:
# EJERCICIO: implementar cuatro llamadas de TEST de 'filtra_por_años':
#   - Número de películas con fecha posterior a 2019
#   - Número de películas del género 'Drama'
#   - Número de películas que incluyan la palabra 'man' en el título
#   - Número de películas con popularidad mayor que 1

# Las salidas esperadas para estas consultas son:
#  7336
#   318
#  8034

print(len(filtro_generico(REGISTROS, lambda x:x.fecha.year>=2020)))
print(len(filtro_generico(REGISTROS, lambda x:'Drama' in x.generos)))
print(len(filtro_generico(REGISTROS, lambda x:'man' in x.titulo)))
print(len(filtro_generico(REGISTROS, lambda x: x.popularidad > 1)))

La siguiente función se llama <code>agrupa_por_genero</code>. Toma como entrada una lista de registros, y produce como salida un diccionario cuyas claves son los generos, y los valores son los registros correspondientes a dicho género.

In [ ]:
def agrupa_por_genero(registros):
    ''' Crea un diccionario con los registros por cada género
    
    ENTRADA:
       - registros: lista de registros -> [Pelicula(str, str, float, datetime)]
    SALIDA:
       - diccionario con los registros de cada género -> {str: [Pelicula(str, str, float, datetime)]}
    '''
    generos = calcula_generos(registros)
    agrupacion = dict()
    for genero in generos:
        agrupacion[genero] = [r for r in registros if genero in r.generos]
    return agrupacion

In [ ]:
# TEST  de 'agrupa_por_genero', la salida esperada es:
#    Foreign -> 52
#    TV Movie -> 272
#    Western -> 332
#    Documentary -> 336
#    Music -> 478
#    War -> 488
#    History -> 571
#    Mystery -> 1106
#    Animation -> 1170
#    Fantasy -> 1308
#    Science Fiction -> 1544
#    Family -> 1568
#    Horror -> 1962
#    Crime -> 1987
#    Adventure -> 2172
#    Romance -> 2723
#    Action -> 3527
#    Thriller -> 3857
#    Comedy -> 5094
#    Drama -> 7336

grupos_genero = agrupa_por_genero(REGISTROS)
generos = sorted(grupos_genero, key=lambda x:len(grupos_genero.get(x)))

for g in generos:
    print(g, '->', len(grupos_genero[g]))

## 3. Visualización (2 puntos)

La función de visualización que implementaremos será <code>muestra_peliculas_mas_populares_de_genero</code>. Toma como entrada un diccionario de registros agrupados por género, y muestra un diagrama de barras con el ranking de las películas más populares de un determinado género.

In [ ]:
def muestra_peliculas_mas_populares_de_genero(grupos_genero, genero, top=10):
    ''' Genera un diagrama de barras con el ranking de las películas más populares de un determinado género
        
    ENTRADA:
        - grupos_genero: registros agrupados por géneros -> {str: [Pelicula(str, str, float, datetime)]}
        - genero: para el que se genera la gráfica -> str
        - top: número de películas a incluir en el ranking (por defecto 10) -> int
    SALIDA EN PANTALLA:
        - diagrama de barras con el ranking de las películas más populares del género
           
    Se usarán las siguientes instrucciones matplotlib para generar la gráfica:
            plt.barh(peliculas, popularidades)
            plt.show()
            
    Donde las dos variables usadas significan lo siguiente:
        - titulos: lista (ordenada de mayor a menor popularidad) de las películas a mostrar
        - popularidades: lista (alineada con la anterior) con los valores de popularidad
    '''
    peliculas = sorted(grupos_genero[genero], reverse=True, key=lambda r:r.popularidad)[:top]
    titulos = [p.titulo for p in peliculas]
    popularidades = [p.popularidad for p in peliculas]
    plt.barh(titulos, popularidades)
    plt.show()

La salida de la primera celda de test debería ser la siguiente:

<img src="./img/test1.png" alt="Drawing" style="width:8cm;"/>

In [ ]:
grupo_comedy = {'Comedy': 
          [Pelicula('The Emoji Movie', 'Comedy|Family|Animation', 10.83876, datetime(2017, 7, 28, 0, 0)),
           Pelicula('Minions', 'Family|Animation|Adventure|Comedy', 156.093026, datetime(2015, 6, 17, 0, 0)), 
           Pelicula('Despicable Me 3', 'Science Fiction|Adventure|Animation|Comedy|Family', 79.053164, datetime(2017, 6, 15, 0, 0)), 
           Pelicula('Deadpool', 'Action|Adventure|Comedy|Romance', 32.101662, datetime(2016, 2, 9, 0, 0))]
     }

muestra_peliculas_mas_populares_de_genero(grupo_comedy, 'Comedy', top=3)

La salida de la segunda celda de test debería ser la siguiente:

<img src="./img/test2.png" alt="Drawing" style="width:14cm;"/>

In [ ]:
# TEST de 'muestra_peliculas_mas_populares_de_genero'
grupos_genero = agrupa_por_genero(REGISTROS)
muestra_peliculas_mas_populares_de_genero(grupos_genero, 'Comedy')